In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import io
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds

from tensorflow import keras
from tensorflow.keras import layers

c:\Users\klikh\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Make sure we don't get any GPU errors
# physical_devices = tf.config.list_physical_devices("GPU")
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
(ds_train, ds_test), ds_info = tfds.load(
    "cifar10",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)


In [4]:
def normalize_img(image, label):
    """Normalizes images"""
    return tf.cast(image, tf.float32) / 255.0, label

In [5]:
def augment(image, label):
    if tf.random.uniform((), minval=0, maxval=1) < 0.1:
        image = tf.tile(tf.image.rgb_to_grayscale(image), [1, 1, 3])

    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_flip_left_right(image)

    return image, label

In [6]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

In [7]:
# Setup for train dataset
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.map(augment)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

In [8]:
# Setup for test Dataset
ds_test = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_train.batch(BATCH_SIZE)
ds_test = ds_train.prefetch(AUTOTUNE)

In [9]:
class_names = [
    "Airplane",
    "Autmobile",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Ship",
    "Truck",
]


In [10]:

def get_model():
    model = keras.Sequential(
        [
            layers.Input((32, 32, 3)),
            layers.Conv2D(8, 3, padding="same", activation="relu"),
            layers.Conv2D(16, 3, padding="same", activation="relu"),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dense(64, activation="relu"),
            layers.Dropout(0.1),
            layers.Dense(10),
        ]
    )

    return model


In [11]:
model = get_model()

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)


In [12]:
tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir="logs", histogram_freq=1,
)

- it is the most easiest way to use tensorboard
- histogram_frequency =1: distribution plot for gradients e.t.c

In [13]:
model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_test,
    callbacks=[tensorboard_callback],
    verbose=2,
)

Epoch 1/5
1563/1563 - 49s - loss: 1.6095 - accuracy: 0.4230 - val_loss: 1.3332 - val_accuracy: 0.5278 - 49s/epoch - 32ms/step
Epoch 2/5
1563/1563 - 41s - loss: 1.3069 - accuracy: 0.5338 - val_loss: 1.1339 - val_accuracy: 0.5984 - 41s/epoch - 26ms/step
Epoch 3/5
1563/1563 - 42s - loss: 1.1931 - accuracy: 0.5763 - val_loss: 1.0650 - val_accuracy: 0.6234 - 42s/epoch - 27ms/step
Epoch 4/5
1563/1563 - 43s - loss: 1.1193 - accuracy: 0.6030 - val_loss: 0.9936 - val_accuracy: 0.6490 - 43s/epoch - 27ms/step
Epoch 5/5
1563/1563 - 42s - loss: 1.0590 - accuracy: 0.6254 - val_loss: 0.9545 - val_accuracy: 0.6675 - 42s/epoch - 27ms/step
